In [ ]:
# Imports
import os
import gc
import cv2
import pandas as pd
from src.sam2_eval_utils import (
    initialize_model,
    load_image,
    load_mask,
    select_point_prompt,
    save_predicted_mask,
    process_image,
    evaluate_organ
)

# Define dataset paths
base_dir = os.getcwd()  # Use the current working directory for base_dir
base_dataset_path = os.path.join(base_dir, '../Datasets')
dataset_name = "Endoscapes"  

frames_path = os.path.join(base_dataset_path, dataset_name, "test", "Frames")
masks_path = os.path.join(base_dataset_path, dataset_name, "test", "Masks")
predicted_masks_path = os.path.join(base_dataset_path, dataset_name, "test", "PredictedMasks")
os.makedirs(predicted_masks_path, exist_ok=True)

# Temporary directory for JPEG conversion
temp_jpg_folder = os.path.join(base_dir, 'temp_jpg_frames')
os.makedirs(temp_jpg_folder, exist_ok=True)

def convert_to_mp4(image_path, temp_folder):
    """Convert a single image to an MP4 video format and store in a temporary folder."""
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    fps = 1  # Frame per second; adjust based on your needs
    video_path = os.path.join(temp_folder, os.path.splitext(os.path.basename(image_path))[0] + '.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify codec
    video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

    video_writer.write(image)

    video_writer.release()
    
    return video_path

# Initialize the SAM2 model
sam_model = initialize_model()

In [ ]:
# Function to find all mask files with full paths
def find_all_files(root_dir, file_extension=(".png", ".jpg")):
    """Recursively find all files with the specified extension in a directory."""
    mask_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(file_extension):
                mask_files.append(os.path.join(dirpath, filename))
    return mask_files

# Function to create a map of frames by sub-path (ignores class folder structure in masks)
def create_frame_map(frames_dir, file_extension=(".png", ".jpg")):
    """Map frame files by their sub-paths, ignoring organ class structure in masks."""
    frame_map = {}
    for dirpath, _, filenames in os.walk(frames_dir):
        for filename in filenames:
            if filename.endswith(file_extension):
                relative_sub_path = os.path.relpath(os.path.join(dirpath, filename), frames_path)
                relative_sub_path_no_ext = os.path.splitext(relative_sub_path)[0]
                frame_map[relative_sub_path_no_ext] = os.path.join(dirpath, filename)
    return frame_map

# Run inference
def run_inference():
    print(f"\nProcessing dataset: {dataset_name}")

    # Create a map of all frames by sub-path
    frame_map = create_frame_map(frames_path)
    print("Frame map created with entries:", len(frame_map))

    # Get all mask files in the masks directory
    mask_files = find_all_files(masks_path)
    print("Mask files created with entries:", len(mask_files))

    # Loop through each mask file
    for mask_path in mask_files:
        # Load the mask image
        mask_image = load_mask(mask_path)
        if mask_image is None:
            print(f"Failed to load mask: {mask_path}")
            continue

        # Determine sub-path from the mask path, ignoring the class folder
        relative_mask_path = os.path.relpath(mask_path, masks_path)
        organ_class_folder = relative_mask_path.split(os.sep)[0]
        mask_sub_path = os.path.relpath(mask_path, os.path.join(masks_path, organ_class_folder))
        mask_sub_path_no_ext = os.path.splitext(mask_sub_path)[0]

        # Retrieve the corresponding frame path from the map
        frame_path = frame_map.get(mask_sub_path_no_ext)
        
        if frame_path is None:
            print(f"No matching frame found for mask: {mask_path}")
            continue

        # Convert frame to MP4 if it’s a PNG or JPG and store in temp folder
        if frame_path.endswith('.png') or frame_path.endswith('.jpg'):
            frame_path_mp4 = convert_to_mp4(frame_path, temp_jpg_folder)
        #else:
        #    frame_path_mp4 = frame_path  # No conversion needed if already in MP4 format

        # Load the frame image
        frame_image = load_image(frame_path)
        if frame_image is None:
            print(f"Failed to load frame: {frame_path}")
            continue

        print(f"Processing Frame: {frame_path} with Mask: {mask_path}")

        # Initialize SAM2 inference state
        inference_state = sam_model.init_state(frame_path_mp4)

        # Sample points from the mask
        sampled_points, labels = select_point_prompt(mask_image)
        if sampled_points is None:
            print(f"No valid points found in mask: {mask_path}")
            continue

        # Create path for predicted mask output
        # Create the predicted mask path, retaining the full directory structure
        predicted_mask_relative_path = os.path.relpath(mask_path, masks_path)  # Retain the full path from Masks folder
        predicted_mask_path = os.path.join(predicted_masks_path, predicted_mask_relative_path)
        os.makedirs(os.path.dirname(predicted_mask_path), exist_ok=True)

        # Perform inference
        process_image(sam_model, inference_state, frame_image, mask_image, sampled_points, labels, predicted_mask_path, idx=0)
        
        # Reset SAM2 model state
        sam_model.reset_state(inference_state)
        gc.collect()

        # Delete the temporary MP4 file
        if os.path.exists(frame_path_mp4):
            os.remove(frame_path_mp4)
            print(f"Deleted temporary file: {frame_path_mp4}")

    print("Segmentation processing completed for the dataset.")

# Execute the inference
run_inference()


Processing dataset: Endoscapes
Frame map created with entries: 74
Mask files created with entries: 238
Processing Frame: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\Frames\165_23650.jpg with Mask: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\Masks\Cystic Artery\165_23650.png
Total prompt points found in mask: 1500
Formatted points: [[469, 329]]
Starting inference on image index 0
Running SAM2 model inference on frame 0 with object ID 1
Inference successful; generating predicted mask for output path c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\PredictedMasks\Cystic Artery\165_23650.png
Predicted mask successfully saved at: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\PredictedMasks\Cystic Artery\165_23650.png
Failed to save predicted mask at c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\

C:\Users\devan\Downloads\SAM2_Eval\Finetuning\segment-anything-2\sam2\sam2_video_predictor.py:961: UserWarning: cannot import name '_C' from 'sam2' (C:\Users\devan\Downloads\SAM2_Eval\Finetuning\segment-anything-2\sam2\__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


Processing Frame: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\Frames\165_24400.jpg with Mask: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\Masks\Cystic Artery\165_24400.png
Total prompt points found in mask: 1826
Formatted points: [[550, 248]]
Starting inference on image index 0
Running SAM2 model inference on frame 0 with object ID 1
Inference successful; generating predicted mask for output path c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\PredictedMasks\Cystic Artery\165_24400.png
Predicted mask successfully saved at: c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\PredictedMasks\Cystic Artery\165_24400.png
Failed to save predicted mask at c:\Users\devan\Downloads\SAM2_Eval\SAM2_Eval_Surgical_Datasets\../Datasets\Endoscapes\test\PredictedMasks\Cystic Artery\165_24400.png
Processing Frame: c:\Users\devan\

In [2]:
def evaluate_all_organs():
    """Evaluate predicted and ground truth masks across all organs and compile results."""
    all_results = []

    # Ensure both Masks and PredictedMasks paths exist
    if not os.path.exists(masks_path) or not os.path.exists(predicted_masks_path):
        print("Error: Either 'Masks' or 'PredictedMasks' directory is missing.")
        return pd.DataFrame()  # Return an empty DataFrame if paths are incorrect

    # Iterate over each organ folder within the Masks directory
    organ_folders = os.listdir(masks_path)
    
    for organ_folder in organ_folders:
        # Paths specific to each organ in Masks and PredictedMasks
        organ_masks_path = os.path.join(masks_path, organ_folder)
        organ_predicted_masks_path = os.path.join(predicted_masks_path, organ_folder)

        # Check if corresponding PredictedMasks folder exists for the organ
        if not os.path.exists(organ_predicted_masks_path):
            print(f"Error: PredictedMasks folder missing for organ: {organ_folder}")
            continue

        # Evaluate metrics for this organ
        organ_results = evaluate_organ(organ_predicted_masks_path, organ_masks_path)
        
        if organ_results:
            organ_df = pd.DataFrame(organ_results)
            all_results.append({
                "Organ": organ_folder,
                "Mean IoU": organ_df['IoU'].mean(),
                "Std IoU": organ_df['IoU'].std(),
                "Mean Dice": organ_df['Dice'].mean(),
                "Std Dice": organ_df['Dice'].std(),
                "Mean Precision": organ_df['Precision'].mean(),
                "Std Precision": organ_df['Precision'].std(),
                "Mean Recall": organ_df['Recall'].mean(),
                "Std Recall": organ_df['Recall'].std()
            })
        else:
            print(f"No valid comparisons found for organ: {organ_folder}")

    # Compile and display results across all organs
    all_results_df = pd.DataFrame(all_results)
    print(all_results_df)

    return all_results_df

# Run evaluation with the paths defined in the original setup
evaluate_all_organs()

                   Organ  Mean IoU   Std IoU  Mean Dice  Std Dice  \
0          Cystic Artery  0.141240  0.209779   0.204935  0.239556   
1            Cystic Duct  0.192079  0.219518   0.277302  0.253537   
2           Cystic Plate  0.224233  0.282786   0.299765  0.308746   
3           Gall Bladder  0.432537  0.279206   0.550806  0.280412   
4  Hepatocystic triangle  0.611310  0.242618   0.729723  0.227155   
5            Instruments  0.599342  0.317335   0.689573  0.311756   

   Mean Precision  Std Precision  Mean Recall  Std Recall  
0        0.165264       0.264890     0.942401    0.120248  
1        0.238688       0.263809     0.774304    0.330633  
2        0.304620       0.371880     0.849211    0.266698  
3        0.679110       0.309825     0.694919    0.356773  
4        0.756315       0.285672     0.839404    0.213958  
5        0.911509       0.185592     0.629441    0.324479  


,Organ,Mean IoU,Std IoU,Mean Dice,Std Dice,Mean Precision,Std Precision,Mean Recall,Std Recall
0,Cystic Artery,0.141240,0.209779,0.204935,0.239556,0.165264,0.264890,0.942401,0.120248
1,Cystic Duct,0.192079,0.219518,0.277302,0.253537,0.238688,0.263809,0.774304,0.330633
2,Cystic Plate,0.224233,0.282786,0.299765,0.308746,0.304620,0.371880,0.849211,0.266698
3,Gall Bladder,0.432537,0.279206,0.550806,0.280412,0.679110,0.309825,0.694919,0.356773
4,Hepatocystic triangle,0.611310,0.242618,0.729723,0.227155,0.756315,0.285672,0.839404,0.213958
5,Instruments,0.599342,0.317335,0.689573,0.311756,0.911509,0.185592,0.629441,0.324479
